In [ ]:
# %pip install --upgrade datasets groq

In [ ]:
import requests
import os
from datasets import load_dataset

# Config

In [ ]:
INPUT_DATASET = "dariolopez/justicio-BOE-A-1978-31229-constitucion-by-articles-qa"

In [ ]:
SYSTEM_PROMPT = """
Como un experto en derecho y leyes españolas, tu tarea es responder preguntas sobre el Boletín Oficial del Estado (BOE) de España. Para ello, debes tener en cuenta y utilizar el contexto proporcionado para responder de forma precisa a la pregunta del usuario.
Asegúrate de responder siempre en español. Si no conoces la respuesta o no tienes suficiente información para responderla, simplemente admítelo; no intentes inventar una respuesta.
Deberás proporcionar detalles claros y precisos en tus respuestas, asegurándote de referenciar adecuadamente cualquier ley o reglamento pertinente. Tu objetivo es proporcionar respuestas útiles y precisas para ayudar a los usuarios a entender mejor el BOE y cómo se aplica a sus preguntas.
"""

In [ ]:
SYSTEM_PROMPT_CONTEXT = """
El contexto tiene un formato de lista, donde cada elemento será un diccionario con dos claves:
[{'context': 'contexto necesario para contestar la pregunta', 'score': 0.8}]
La clave 'context' contendrá la información necesaria para contestar a la pregunta y la clave 'score' será una puntuación de entre 0.0 y 1.0. Deberás dar más importancia al contexto cuanto mayor sea el score.
En la respuesta no menciones nada sobre el contexto o los scores.
"""

In [ ]:
URL = "http://localhost:5001/semantic_search"

In [ ]:
dataset = load_dataset(INPUT_DATASET)

In [ ]:
from groq import Groq
import json
import time


client = Groq(
    api_key=os.environ.get('GROQ_API_KEY'),
)

In [ ]:
answers_groq = []
contexts = []

for idx, row in enumerate(dataset['train']):
    time.sleep(20)  # groq limits: https://console.groq.com/settings/limits
    try:
        docs = requests.get(URL, params={'input_query': row['question'], 'collection_name': 'boe-bge-m3'}, timeout=60)
        context_preprocessed = [{"context": doc[0]['page_content'], "score": doc[1]} for doc in docs.json()]
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {
                "role": "system",
                "content": SYSTEM_PROMPT_CONTEXT,
            },
            {"role": "system", "content": "A continuación se proporciona el contexto:"},
            {"role": "system", "content": str(context_preprocessed)},
            {
                "role": "system",
                "content": "A continuación se proporciona la pregunta del usuario:",
            },
            {"role": "user", "content": row['question']},
        ]
        chat_completion = client.chat.completions.create(
            messages=messages,
            model="llama3-70b-8192",
            temperature=0,
            max_tokens=2048,
            stream=False,
        )
        answer = chat_completion.choices[0].message.content
    
    except Exception as e:
        print(e)
        answer = None

    answers_groq.append(answer)
    contexts.append(context_preprocessed)
    print(f"{idx+1} - {row['question']}")
    print(row['answer'])
    print(answer)
    print("\n\n")

In [ ]:
len(answers_groq)

In [ ]:
len(contexts)

In [ ]:
dataset

In [ ]:
dataset['train'] = dataset['train'].add_column('context_qa', contexts)
dataset['train'] = dataset['train'].add_column('response_groq_llama3_70b_8192', answers_groq)

In [ ]:
dataset

In [ ]:
import huggingface_hub

huggingface_hub.login()

In [ ]:
output_dataset = f"{INPUT_DATASET}-bge-m3-groq_llama3_70b_8192"

In [ ]:
dataset.push_to_hub(output_dataset)

In [ ]:
output_dataset